# Time Spend on Each Step and Error Rates (Step Repetition)

In [181]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

%matplotlib inline

In [184]:
# Dataset:
df = pd.read_csv('../data/clean/df_final_experiment_web_data_demo.csv', sep=',')
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5,64,79.0,U,2,189023.86,1,4
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5,64,79.0,U,2,189023.86,1,4
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5,64,79.0,U,2,189023.86,1,4
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5,64,79.0,U,2,189023.86,1,4
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5,64,79.0,U,2,189023.86,1,4


In [186]:
df['date_time'].dtype

dtype('O')

In [188]:
# Sort the DataFrame by 'date_time' to see each session chronologically 
df = df.sort_values(by='date_time', ascending=True)
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
34571,7179755,167765295_97487764427,264484508_5982901710_928530,start,2017-03-15 00:19:28,Control,28,343,67.5,M,3,352322.35,2,5
34570,7179755,167765295_97487764427,264484508_5982901710_928530,step_1,2017-03-15 00:20:50,Control,28,343,67.5,M,3,352322.35,2,5
34569,7179755,167765295_97487764427,264484508_5982901710_928530,step_2,2017-03-15 00:22:52,Control,28,343,67.5,M,3,352322.35,2,5
34568,7179755,167765295_97487764427,264484508_5982901710_928530,step_3,2017-03-15 00:23:47,Control,28,343,67.5,M,3,352322.35,2,5
34567,7179755,167765295_97487764427,264484508_5982901710_928530,confirm,2017-03-15 00:26:24,Control,28,343,67.5,M,3,352322.35,2,5


In [190]:
df.shape

(317235, 14)

## Time Spent on Each Step and Step Repetition (Error rate) in Session Analysis

This analysis aims to determine which step in the process is most prone to errors, as evidenced by users repeatedly revisiting that specific step. This indicates potential confusion about what needs to be done at that step, or that the user interface (UI) may be problematic. 

We will identify which step takes the most time for users in both the Control and Test groups (Time Spent on Each Step), keeping in mind that users do not necessarily follow the steps in a strict sequence. Instead, they may return to previous steps, move forward, and then go back again as many times as needed until they reach the final confirmation step.

Additionally, the system records user activity at regular intervals, which means there are instances where the same step appears repeatedly in the records. This indicates that the user remains on the same step without moving to another, and it is just another entry in the log. With this clear, we sum the time spent on the same step as long as the user doesn't move to a different step.

_Agreement on Data Handling:_
The final step in the process is 'confirm'. Therefore, we will not consider any records that occur after the user reaches the 'confirm' step. We assume that if a user continues to move between steps after achieving the last step, it is because they are exploring further or seeking more details. However, the primary objective to complete the process has been achieved.

To facilitate this analysis, we have divided the data into two groups:
- Successful Cases: sessions where all steps are completed (presence of both a 'start' and a 'confirm' in the same session). 
- Failed Cases: sessions where users fail to complete the process during a session.


### Successful Cases: all steps are completed in the same session
#### Time Spent on Each Step

In [60]:
# Step 1: Filter for users who have completed all steps (start, step_1, step_2, step_3, confirm)
required_steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Identify sessions (visit_id) that have all the required steps
completed_sessions = df.groupby('visit_id')['process_step'].apply(lambda x: set(required_steps).issubset(set(x)))

# Filter the dataframe to only include completed sessions
df_completed = df[df['visit_id'].isin(completed_sessions[completed_sessions].index)]

df_completed.head(14)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
34571,7179755,167765295_97487764427,264484508_5982901710_928530,start,2017-03-15 00:19:28,Control,28,343,67.5,M,3,352322.35,2,5
34570,7179755,167765295_97487764427,264484508_5982901710_928530,step_1,2017-03-15 00:20:50,Control,28,343,67.5,M,3,352322.35,2,5
34569,7179755,167765295_97487764427,264484508_5982901710_928530,step_2,2017-03-15 00:22:52,Control,28,343,67.5,M,3,352322.35,2,5
34568,7179755,167765295_97487764427,264484508_5982901710_928530,step_3,2017-03-15 00:23:47,Control,28,343,67.5,M,3,352322.35,2,5
34567,7179755,167765295_97487764427,264484508_5982901710_928530,confirm,2017-03-15 00:26:24,Control,28,343,67.5,M,3,352322.35,2,5
204723,9584408,748244138_48778380454,484298588_81471639218_981974,start,2017-03-15 06:03:24,Control,7,87,48.5,U,2,71834.25,2,5
204722,9584408,748244138_48778380454,484298588_81471639218_981974,step_1,2017-03-15 06:03:44,Control,7,87,48.5,U,2,71834.25,2,5
204721,9584408,748244138_48778380454,484298588_81471639218_981974,step_2,2017-03-15 06:03:53,Control,7,87,48.5,U,2,71834.25,2,5
204720,9584408,748244138_48778380454,484298588_81471639218_981974,step_3,2017-03-15 06:06:10,Control,7,87,48.5,U,2,71834.25,2,5
204719,9584408,748244138_48778380454,484298588_81471639218_981974,start,2017-03-15 06:14:28,Control,7,87,48.5,U,2,71834.25,2,5


In [64]:
df_completed.shape

(200657, 14)

In [66]:
# Step 2: Calculate time spent on each step
# Ensure the dataset is sorted by 'visit_id' and 'date_time' (string)
df_completed = df_completed.sort_values(by=['visit_id', 'date_time'], ascending=[True, True])

df_completed

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
235345,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,Test,7,88,23.5,M,2,26436.73,6,9
235344,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,Test,7,88,23.5,M,2,26436.73,6,9
235343,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,Test,7,88,23.5,M,2,26436.73,6,9
235342,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:35,Test,7,88,23.5,M,2,26436.73,6,9
235341,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:41,Test,7,88,23.5,M,2,26436.73,6,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90533,4064969,110735946_42614968419,999985675_64610694964_443659,start,2017-04-20 09:45:18,Control,6,75,64.5,U,2,74783.76,5,8
90532,4064969,110735946_42614968419,999985675_64610694964_443659,step_1,2017-04-20 09:46:26,Control,6,75,64.5,U,2,74783.76,5,8
90531,4064969,110735946_42614968419,999985675_64610694964_443659,step_2,2017-04-20 09:47:04,Control,6,75,64.5,U,2,74783.76,5,8
90530,4064969,110735946_42614968419,999985675_64610694964_443659,step_3,2017-04-20 09:48:45,Control,6,75,64.5,U,2,74783.76,5,8


In [68]:
# Step 2: Sort by visit_id and date_time to maintain chronological order within sessions
df_completed = df_completed.sort_values(by=['visit_id', 'date_time'], ascending=[True, True])
df_completed.head(30)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
235345,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,Test,7,88,23.5,M,2,26436.73,6,9
235344,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,Test,7,88,23.5,M,2,26436.73,6,9
235343,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,Test,7,88,23.5,M,2,26436.73,6,9
235342,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:35,Test,7,88,23.5,M,2,26436.73,6,9
235341,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:41,Test,7,88,23.5,M,2,26436.73,6,9
235340,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:21:45,Test,7,88,23.5,M,2,26436.73,6,9
235339,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:21:59,Test,7,88,23.5,M,2,26436.73,6,9
235338,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:22:04,Test,7,88,23.5,M,2,26436.73,6,9
235337,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:22:08,Test,7,88,23.5,M,2,26436.73,6,9
235336,7338123,612065484_94198474375,100019538_17884295066_43909,step_3,2017-04-09 16:24:01,Test,7,88,23.5,M,2,26436.73,6,9


In [70]:
# Step 3: Remove any records that occur after the user reaches the 'confirm' step
# We first find the index of the first occurrence of 'confirm' for each visit_id
df_completed['confirm_reached'] = df_completed.groupby('visit_id')['process_step'].transform(lambda x: (x == 'confirm').cumsum())

# Remove any records after 'confirm' is reached (including multiple 'confirm' rows)
df_completed = df_completed[df_completed['confirm_reached'] <= 1]

In [72]:
df_completed.shape

(196645, 15)

In [74]:
df_completed.head(30)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,confirm_reached
235345,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,Test,7,88,23.5,M,2,26436.73,6,9,0
235344,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,Test,7,88,23.5,M,2,26436.73,6,9,0
235343,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,Test,7,88,23.5,M,2,26436.73,6,9,0
235342,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:35,Test,7,88,23.5,M,2,26436.73,6,9,0
235341,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:41,Test,7,88,23.5,M,2,26436.73,6,9,0
235340,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:21:45,Test,7,88,23.5,M,2,26436.73,6,9,0
235339,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:21:59,Test,7,88,23.5,M,2,26436.73,6,9,0
235338,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:22:04,Test,7,88,23.5,M,2,26436.73,6,9,0
235337,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:22:08,Test,7,88,23.5,M,2,26436.73,6,9,0
235336,7338123,612065484_94198474375,100019538_17884295066_43909,step_3,2017-04-09 16:24:01,Test,7,88,23.5,M,2,26436.73,6,9,0


In [76]:
# Step 4: Calculate time difference between consecutive rows (without converting 'date_time')
def time_difference(row1, row2):
    time_format = "%Y-%m-%d %H:%M:%S"
    # Parse the string as a datetime object
    t1 = datetime.strptime(row1, time_format)
    t2 = datetime.strptime(row2, time_format)
    # Calculate the time difference in seconds
    return (t2 - t1).total_seconds()


# Create a column to hold the next step's date_time
df_completed['next_date_time'] = df_completed.groupby('visit_id')['date_time'].shift(-1)


# Calculate the time spent on each step
df_completed['time_spent'] = df_completed.apply(
    lambda row: time_difference(row['date_time'], row['next_date_time']) if pd.notnull(row['next_date_time']) else 0,
    axis=1
)

# Step 5: Sum the time for repeated steps
# If the user stays on the same process_step, sum the time differences
df_completed['next_process_step'] = df_completed.groupby('visit_id')['process_step'].shift(-1)


# Initialize a column to track cumulative time for repeated steps
df_completed['cumulative_time_spent'] = df_completed['time_spent']


# Sum time for repeated steps (if the next process_step is the same as the current one)
df_completed['cumulative_time_spent'] = df_completed.groupby(['visit_id', 'process_step'])['cumulative_time_spent'].cumsum()


# If the user moves to a different step, take the final cumulative time for that step
df_completed['final_time_spent'] = df_completed.groupby('visit_id')['cumulative_time_spent'].shift(-1)
df_completed.loc[df_completed['process_step'] != df_completed['next_process_step'], 'final_time_spent'] = df_completed['cumulative_time_spent']


# Step 6: Set the time spent on 'confirm' to 0 since it's the last step
df_completed.loc[df_completed['process_step'] == 'confirm', 'final_time_spent'] = 0


# Display the first few rows to verify the results
df_completed.head(17)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,confirm_reached,next_date_time,time_spent,next_process_step,cumulative_time_spent,final_time_spent
235345,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:21:12,16.0,step_1,16.0,16.0
235344,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:21:21,9.0,step_2,9.0,9.0
235343,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:21:35,14.0,step_1,14.0,14.0
235342,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:35,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:21:41,6.0,step_1,15.0,19.0
235341,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:41,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:21:45,4.0,start,19.0,19.0
235340,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:21:45,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:21:59,14.0,start,30.0,35.0
235339,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:21:59,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:22:04,5.0,step_1,35.0,35.0
235338,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:22:04,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:22:08,4.0,step_2,23.0,23.0
235337,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:22:08,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:24:01,113.0,step_3,127.0,127.0
235336,7338123,612065484_94198474375,100019538_17884295066_43909,step_3,2017-04-09 16:24:01,Test,7,88,23.5,M,2,26436.73,6,9,0,2017-04-09 16:24:58,57.0,confirm,57.0,57.0


In [78]:
# export the dataframe to a CSV file
df_completed.to_csv('../data/clean/df_completed.csv', index=False)

In [80]:
# Step 7: Group by Variation and process_step to calculate the average time spent per step
avg_time_spent_per_step = df_completed.groupby(['Variation', 'process_step'])['final_time_spent'].mean().reset_index()

# Display the results
display(avg_time_spent_per_step)

,Variation,process_step,final_time_spent
0,Control,confirm,0.000000
1,Control,start,106.486414
2,Control,step_1,71.125365
3,Control,step_2,119.004766
4,Control,step_3,165.731749
5,Test,confirm,0.000000
6,Test,start,105.480604
7,Test,step_1,97.053191
8,Test,step_2,118.236380
9,Test,step_3,152.913187


In [82]:
# Convert final_time_spent from seconds to a formatted string of minutes and seconds
def format_time(seconds):
    # Convert seconds to minutes and seconds
    minutes = int(seconds // 60)
    remaining_seconds = int(seconds % 60)
    return f"{minutes}m {remaining_seconds}s"

# Apply the time formatting to the final_time_spent column
avg_time_spent_per_step['final_time_spent_formatted'] = avg_time_spent_per_step['final_time_spent'].apply(format_time)
display(avg_time_spent_per_step)

,Variation,process_step,final_time_spent,final_time_spent_formatted
0,Control,confirm,0.000000,0m 0s
1,Control,start,106.486414,1m 46s
2,Control,step_1,71.125365,1m 11s
3,Control,step_2,119.004766,1m 59s
4,Control,step_3,165.731749,2m 45s
5,Test,confirm,0.000000,0m 0s
6,Test,start,105.480604,1m 45s
7,Test,step_1,97.053191,1m 37s
8,Test,step_2,118.236380,1m 58s
9,Test,step_3,152.913187,2m 32s


In [84]:
# Define the correct order for process steps for better visualization in the table
step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Sort the avg_time_spent_per_step DataFrame by process_step using the defined order
avg_time_spent_per_step['process_step'] = pd.Categorical(avg_time_spent_per_step['process_step'], categories=step_order, ordered=True)

# Now sort the DataFrame based on this categorical order
avg_time_spent_per_step = avg_time_spent_per_step.sort_values(by='process_step')

display(avg_time_spent_per_step)

,Variation,process_step,final_time_spent,final_time_spent_formatted
1,Control,start,106.486414,1m 46s
6,Test,start,105.480604,1m 45s
2,Control,step_1,71.125365,1m 11s
7,Test,step_1,97.053191,1m 37s
3,Control,step_2,119.004766,1m 59s
8,Test,step_2,118.236380,1m 58s
4,Control,step_3,165.731749,2m 45s
9,Test,step_3,152.913187,2m 32s
0,Control,confirm,0.000000,0m 0s
5,Test,confirm,0.000000,0m 0s


In [86]:
# Step 8: Pivot the table to easily compare Control and Test variations
comparison = avg_time_spent_per_step.pivot(index='process_step', columns='Variation', values=['final_time_spent_formatted', 'final_time_spent'])

# Calculate the difference in time between Control and Test in seconds
comparison['difference_in_seconds'] = comparison[('final_time_spent', 'Test')] - comparison[('final_time_spent', 'Control')]

# Display the comparison between Control and Test with the difference in seconds
comparison_with_difference = comparison[['final_time_spent_formatted', 'difference_in_seconds']]

display(comparison_with_difference)

final_time_spent_formatted         difference_in_seconds
Variation                       Control    Test                      
process_step                                                         
start                            1m 46s  1m 45s             -1.005811
step_1                           1m 11s  1m 37s             25.927825
step_2                           1m 59s  1m 58s             -0.768386
step_3                           2m 45s  2m 32s            -12.818562
confirm                           0m 0s   0m 0s                   0.0

#### Key Analysis:
* The total time spent on each step is very similar between the Control and Test groups, with the largest difference seen in step_1.
* __step_1__:
    - Difference: +25.93 seconds (users in the Test group are spending 26 seconds more on step_1 than in the Control group).
    - Interpretation: Users in the Test group are taking significantly more time in step_1. This could indicate potential confusion or difficulty in understanding the interface or tasks at this step. UI errors.
* __step_3__:
    - Difference:  -12.82 seconds (users in the Test group are spending nearly 13 seconds less on step_3 than the Control group).
    - Interpretation:  Users in the Test group are moving through step_3 faster than those in the Control group. This could indicate an improvement in user experience for this step in the Test variation.
 
##### Actionable Recommendations:
- Investigate step_1: The extra 26 seconds that users in the Test group are spending on step_1 suggests there might be confusion or friction. Consider reviewing the UI/UX of this step to identify any barriers.
- Leverage success in step_3: The Test group is performing better in step_3, taking 13 seconds less time. If this improvement is intentional, consider replicating the changes made here to other steps to enhance the user experience further.

### Successful Cases: all steps are completed in the same session
#### Step Repetition in sessions (Error Rate)

In [151]:
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
34571,7179755,167765295_97487764427,264484508_5982901710_928530,start,2017-03-15 00:19:28,Control,28,343,67.5,M,3,352322.35,2,5
34570,7179755,167765295_97487764427,264484508_5982901710_928530,step_1,2017-03-15 00:20:50,Control,28,343,67.5,M,3,352322.35,2,5
34569,7179755,167765295_97487764427,264484508_5982901710_928530,step_2,2017-03-15 00:22:52,Control,28,343,67.5,M,3,352322.35,2,5
34568,7179755,167765295_97487764427,264484508_5982901710_928530,step_3,2017-03-15 00:23:47,Control,28,343,67.5,M,3,352322.35,2,5
34567,7179755,167765295_97487764427,264484508_5982901710_928530,confirm,2017-03-15 00:26:24,Control,28,343,67.5,M,3,352322.35,2,5


In [153]:
df.shape

(317235, 14)

In [155]:
# Step 1: Filter for users who have completed all steps (start, step_1, step_2, step_3, confirm)
required_steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Identify sessions (visit_id) that have all the required steps
completed_sessions = df.groupby('visit_id')['process_step'].apply(lambda x: set(required_steps).issubset(set(x)))

# Filter the dataframe to only include completed sessions
df_completed = df[df['visit_id'].isin(completed_sessions[completed_sessions].index)]

In [157]:
# Step 2: Sort by visit_id and date_time to maintain chronological order within sessions
df_completed = df_completed.sort_values(by=['visit_id', 'date_time'], ascending=[True, True])
df_completed.head(30)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
235345,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,Test,7,88,23.5,M,2,26436.73,6,9
235344,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,Test,7,88,23.5,M,2,26436.73,6,9
235343,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,Test,7,88,23.5,M,2,26436.73,6,9
235342,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:35,Test,7,88,23.5,M,2,26436.73,6,9
235341,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:41,Test,7,88,23.5,M,2,26436.73,6,9
235340,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:21:45,Test,7,88,23.5,M,2,26436.73,6,9
235339,7338123,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:21:59,Test,7,88,23.5,M,2,26436.73,6,9
235338,7338123,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:22:04,Test,7,88,23.5,M,2,26436.73,6,9
235337,7338123,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:22:08,Test,7,88,23.5,M,2,26436.73,6,9
235336,7338123,612065484_94198474375,100019538_17884295066_43909,step_3,2017-04-09 16:24:01,Test,7,88,23.5,M,2,26436.73,6,9


In [159]:
# Step 3: Remove any steps that occur after the **first occurrence** of 'confirm' within each session
def filter_after_confirm(group):
    # Get the index of the first occurrence of 'confirm'
    if 'confirm' in group['process_step'].values:
        confirm_idx = group[group['process_step'] == 'confirm'].index[0]
        # Only keep steps that occur before or at the 'confirm' step
        return group.loc[:confirm_idx]
    return group

df_completed = df_completed.groupby('visit_id', group_keys=False).apply(filter_after_confirm)

/var/folders/t_/k5bhrc1x419crrfg_t8zp5nm0000gp/T/ipykernel_8556/2494258078.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_completed = df_completed.groupby('visit_id', group_keys=False).apply(filter_after_confirm)


In [160]:
# Step 4: Calculate the number of repetitions per step within the same session
step_repetitions = df_completed.groupby(['visit_id', 'process_step']).size().reset_index(name='step_count')

# Step 5: Merge the step repetitions back with the original data to get the Variation for each session
df_step_repetitions = step_repetitions.merge(df[['visit_id', 'Variation']].drop_duplicates(), on='visit_id')

# Step 6: Calculate the average number of repetitions per step for each variation
avg_repetitions = df_step_repetitions.groupby(['Variation', 'process_step'])['step_count'].mean().reset_index(name='avg_step_repetitions')

# Step 7: Sort by process step order
step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
avg_repetitions['process_step'] = pd.Categorical(avg_repetitions['process_step'], categories=step_order, ordered=True)
avg_repetitions = avg_repetitions.sort_values(by='process_step')

# Display the final table
avg_repetitions

,Variation,process_step,avg_step_repetitions
1,Control,start,1.247634
6,Test,start,1.420404
2,Control,step_1,1.191850
7,Test,step_1,1.300062
3,Control,step_2,1.259322
8,Test,step_2,1.186796
4,Control,step_3,1.204593
9,Test,step_3,1.112369
0,Control,confirm,1.000000
5,Test,confirm,1.000000


<br>
<br>
<br>

### Failed Cases: sessions where users abandoned the process before reaching 'confirm'.
#### Time Spent on Each Step

In [163]:
# Dataset:
df = pd.read_csv('../data/clean/df_final_experiment_web_data_demo.csv', sep=',')

# Sort the DataFrame by 'date_time' to see each session chronologically 
df = df.sort_values(by='date_time', ascending=True)
df.head()

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
34571,7179755,167765295_97487764427,264484508_5982901710_928530,start,2017-03-15 00:19:28,Control,28,343,67.5,M,3,352322.35,2,5
34570,7179755,167765295_97487764427,264484508_5982901710_928530,step_1,2017-03-15 00:20:50,Control,28,343,67.5,M,3,352322.35,2,5
34569,7179755,167765295_97487764427,264484508_5982901710_928530,step_2,2017-03-15 00:22:52,Control,28,343,67.5,M,3,352322.35,2,5
34568,7179755,167765295_97487764427,264484508_5982901710_928530,step_3,2017-03-15 00:23:47,Control,28,343,67.5,M,3,352322.35,2,5
34567,7179755,167765295_97487764427,264484508_5982901710_928530,confirm,2017-03-15 00:26:24,Control,28,343,67.5,M,3,352322.35,2,5


In [ ]:
df.shape

In [ ]:
# Step 1: Identify sessions (visit_id) where the user has started but never reached 'confirm'
# First, group by 'visit_id' and extract the set of steps for each session
session_steps = df.groupby('visit_id')['process_step'].apply(set)

# Step 2: Filter to get only sessions that contain 'start' but do not contain 'confirm'
incomplete_sessions = session_steps[session_steps.apply(lambda x: 'start' in x and 'confirm' not in x)]

# Step 3: Filter the original DataFrame to only include those incomplete sessions
df_incomplete = df[df['visit_id'].isin(incomplete_sessions.index)]

# Step 4: Ensure the dataset is sorted by 'visit_id' and 'date_time'
df_incomplete = df_incomplete.sort_values(by=['visit_id', 'date_time'], ascending=[True, True])

# Check the results
df_incomplete.head(15)

In [ ]:
df_incomplete.shape

In [ ]:
# Step 5: Calculate time difference between consecutive rows
def time_difference(row1, row2):
    time_format = "%Y-%m-%d %H:%M:%S"
    # Parse the string as a datetime object
    t1 = datetime.strptime(row1, time_format)
    t2 = datetime.strptime(row2, time_format)
    # Calculate the time difference in seconds
    return (t2 - t1).total_seconds()

# Create a column to hold the next step's date_time
df_incomplete['next_date_time'] = df_incomplete.groupby('visit_id')['date_time'].shift(-1)

# Calculate the time spent on each step
df_incomplete['time_spent'] = df_incomplete.apply(
    lambda row: time_difference(row['date_time'], row['next_date_time']) if pd.notnull(row['next_date_time']) else 0,
    axis=1
)

# Step 6: Sum the time for repeated steps (if user stays on the same step)
df_incomplete['next_process_step'] = df_incomplete.groupby('visit_id')['process_step'].shift(-1)

# Initialize a column to track cumulative time for repeated steps
df_incomplete['cumulative_time_spent'] = df_incomplete['time_spent']

# Sum time for repeated steps (if the next process_step is the same as the current one)
df_incomplete['cumulative_time_spent'] = df_incomplete.groupby(['visit_id', 'process_step'])['cumulative_time_spent'].cumsum()

# If the user moves to a different step, take the final cumulative time for that step
df_incomplete['final_time_spent'] = df_incomplete.groupby('visit_id')['cumulative_time_spent'].shift(-1)
df_incomplete.loc[df_incomplete['process_step'] != df_incomplete['next_process_step'], 'final_time_spent'] = df_incomplete['cumulative_time_spent']

# export the dataframe to a CSV file
df_final.to_csv('../data/clean/df_incompleted.csv', index=False)

df_incomplete.head(13)

In [ ]:
# Step 7: Group by Variation and process_step to calculate the average time spent per step
avg_time_spent_per_step_incomplete = df_incomplete.groupby(['Variation', 'process_step'])['final_time_spent'].mean().reset_index()

display(avg_time_spent_per_step_incomplete)

In [ ]:
# Convert final_time_spent from seconds to a formatted string of minutes and seconds
def format_time(seconds):
    # Convert seconds to minutes and seconds
    minutes = int(seconds // 60)
    remaining_seconds = int(seconds % 60)
    return f"{minutes}m {remaining_seconds}s"

# Apply the time formatting to the final_time_spent column
avg_time_spent_per_step_incomplete['final_time_spent_formatted'] = avg_time_spent_per_step_incomplete['final_time_spent'].apply(format_time)


# Step 9: Pivot the table to easily compare Control and Test variations
comparison_incomplete = avg_time_spent_per_step_incomplete.pivot(index='process_step', columns='Variation', values=['final_time_spent_formatted', 'final_time_spent'])

display(comparison_incomplete)

In [ ]:
# Calculate the difference in time between Control and Test in seconds
comparison_incomplete['difference_in_seconds'] = comparison_incomplete[('final_time_spent', 'Test')] - comparison_incomplete[('final_time_spent', 'Control')]

# Step 10: Display the final table with formatted times and the difference in seconds
comparison_with_difference_incomplete = comparison_incomplete[['final_time_spent_formatted', 'difference_in_seconds']]
comparison_with_difference_incomplete

#### Key Analysis:
* __step_1__: 
    - Users in the Test group are consistently taking more time at step_1, regardless of whether they complete the process. This suggests an issue that needs attention—possibly confusion or complexity at this step. 

* __step_3__:
    - The time difference in step_3 is extremely large for users who abandon the process in the Test group. This indicates a significant bottleneck in the Test version, which may be causing frustration and leading to abandonment.

* There seems to be a strong relationship between step_1 and step_3 for users who did not complete the process, especially in the Test group. Based on the data, users who face difficulties at step_1 tend to struggle even more at step_3, which likely leads to abandonment.
 
##### Actionable Recommendations:
- Simplifying the UI for step_1 in the Test variation. Improving instructions or navigation cues.
- Potential bugs or usability problems with step_3 in the Test variation. Whether there are unclear instructions, poor design, or technical issues at this step
- __Improvement for the Test group__: While step_1 remains a problem for the Test group, the quicker progress in step_3 for those who complete the process is a positive sign. This suggests that some improvements in the Test version are working well at later steps.

<br>
<br>
<br>

### Failed Cases: sessions where users abandoned the process before reaching 'confirm'.
#### Step Repetition in sessions (Error Rate)

In [169]:
df.shape

(317235, 14)

In [173]:
# Step 1: Identify sessions (visit_id) where the user has started but never reached 'confirm'
# First, group by 'visit_id' and extract the set of steps for each session
session_steps = df.groupby('visit_id')['process_step'].apply(set)

# Step 2: Filter to get only sessions that contain 'start' but do not contain 'confirm'
incomplete_sessions = session_steps[session_steps.apply(lambda x: 'start' in x and 'confirm' not in x)]

# Step 3: Filter the original DataFrame to only include those incomplete sessions
df_incomplete = df[df['visit_id'].isin(incomplete_sessions.index)]

# Step 4: Ensure the dataset is sorted by 'visit_id' and 'date_time'
df_incomplete = df_incomplete.sort_values(by=['visit_id', 'date_time'], ascending=[True, True])

# Check the results
df_incomplete.head()

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
234201,105007,676020267_36602664238,100030127_47967100085_936361,start,2017-03-22 11:07:49,Control,9,118,35.0,F,2,34897.47,3,6
26959,5623007,700426174_33289797318,100037962_47432393712_705583,start,2017-04-14 16:41:51,Control,16,202,78.0,M,2,146827.14,5,8
26958,5623007,700426174_33289797318,100037962_47432393712_705583,start,2017-04-14 16:43:16,Control,16,202,78.0,M,2,146827.14,5,8
26957,5623007,700426174_33289797318,100037962_47432393712_705583,step_1,2017-04-14 16:43:55,Control,16,202,78.0,M,2,146827.14,5,8
26956,5623007,700426174_33289797318,100037962_47432393712_705583,start,2017-04-14 16:44:03,Control,16,202,78.0,M,2,146827.14,5,8


In [175]:
df_incomplete.shape

(106703, 14)

In [177]:
# Step 5: Calculate the number of repetitions per step within the same session
step_repetitions_incomplete = df_incomplete.groupby(['visit_id', 'process_step']).size().reset_index(name='step_count')

# Step 6: Merge the step repetitions back with the original data to get the Variation for each session
df_step_repetitions_incomplete = step_repetitions_incomplete.merge(df[['visit_id', 'Variation']].drop_duplicates(), on='visit_id')

# Step 7: Calculate the average number of repetitions per step for each variation
avg_repetitions_incomplete = df_step_repetitions_incomplete.groupby(['Variation', 'process_step'])['step_count'].mean().reset_index(name='avg_step_repetitions')

# Step 8: Sort by process step order
step_order_incomplete = ['start', 'step_1', 'step_2', 'step_3']  # Since 'confirm' is not reached
avg_repetitions_incomplete['process_step'] = pd.Categorical(avg_repetitions_incomplete['process_step'], categories=step_order_incomplete, ordered=True)
avg_repetitions_incomplete = avg_repetitions_incomplete.sort_values(by='process_step')

# Display the final table
avg_repetitions_incomplete

,Variation,process_step,avg_step_repetitions
0,Control,start,1.628993
4,Test,start,1.970089
1,Control,step_1,1.304817
5,Test,step_1,1.465745
2,Control,step_2,1.327555
6,Test,step_2,1.465009
3,Control,step_3,1.304143
7,Test,step_3,1.374003
